# Test random varible evaluation

For first step we create import the sufficient packages 

In [1]:
import sys
import os
import time
sys.path.append('../')
from holoclean import holocleansession , dataset , dataengine
from holoclean.utils import domainpruning
from holoclean.featurization import dcfeaturizer,dc_featurizer

In [2]:
my_path = os.path.abspath(os.path.dirname('__file__'))
path = os.path.join(my_path, "../holoclean/mysql-connector-java-5.1.44-bin.jar")
holoclean_se=holocleansession.HolocleanSession(path)

##  Creating Dataset and dataengine and holocleansession 
In this part we create dataset for keep tracking the runnig step and dataengine for make connector to the database we try to ingest the 10 element csv file and after ingesting it returns df other wise we can read it from database

In [3]:
ds=dataset.Dataset()  
d=dataengine.Dataengine("metadb-config.txt",'datadb-config.txt',ds,holoclean_se.return_sqlcontext())
holoclean_se.set_dataengine(d)
df=d.ingest_spark('1800.csv',holoclean_se.returnspark_session())

index


##  Create featurizer


In [4]:
#dcCode=['t1&t2&EQ(t1.city,t2.city)&EQ(t1.temp,t2.temp)&IQ(t1.tempType,t2.tempType)']
dcCode=['t1&t2&EQ(t1.city,t2.city)&GTE(t1.temp,t2.temp)&IQ(t1.tempType,t2.tempType)']
dcf=dcfeaturizer.DCFeaturizer(df,dcCode,d)

By using mysql we have same table :

In [5]:
start_time_mysql = time.time()

dcf.make_dc_f_table()

#dcf.test()
finish_time_mysql=time.time()
mysqlv=d.get_table_spark("dc_f_mysql")
print(mysqlv.count())
print(mysqlv.show())

811337479035_T
OK!
155
+--------+-------+------------+--------------------+------+
|rv_index|rv_attr|assigned_val|              DCname|tup_id|
+--------+-------+------------+--------------------+------+
|       1|   city| EZE00100082|table1.city=table...|    10|
|       1|   city| ITE00100554|table1.city=table...|     2|
|       1|   city| ITE00100554|table1.city=table...|     7|
|      10|   city| EZE00100082|table1.city=table...|     5|
|       2|   city| EZE00100082|table1.city=table...|     5|
|       3|   city| EZE00100082|table1.city=table...|    10|
|       3|   city| EZE00100082|table1.city=table...|     4|
|       3|   city| EZE00100082|table1.city=table...|     5|
|       3|   city| EZE00100082|table1.city=table...|     9|
|       3|   city| ITE00100554|table1.city=table...|     1|
|       3|   city| ITE00100554|table1.city=table...|     2|
|       3|   city| ITE00100554|table1.city=table...|     6|
|       3|   city| ITE00100554|table1.city=table...|     7|
|       4|   city

Execution time is :

In [6]:
print("--- %s seconds ---" % (finish_time_mysql-start_time_mysql))

--- 0.241194009781 seconds ---


## Deep dive grounding
In this version we create new table which has all initial value and all possible value and by using the create the all wrong violations 

In [7]:
dc_f=dc_featurizer.DCFeaturizer(df,dcCode,d)

we claculate the creation time in this version :

In [8]:
start_time_dd = time.time()
dc_f.make_spark_table(d,holoclean_se.return_sqlcontext())
dc_f.make_dc_f_table()
finish_time_dd=time.time()
mysqlv=d.get_table_spark("dc_f_dd")
print(mysqlv.count())
print(mysqlv.show())

table1.city=table2.city AND table1.temp>=table2.temp AND table1.tempType<>table2.tempType
+--------+------------+-----------+------+--------------------+
|rv_index|assigned_val|    rv_attr|tup_id|             DC_name|
+--------+------------+-----------+------+--------------------+
|       1|        temp|       -125|     7|table1.city=table...|
|       1|        temp|       -135|     7|table1.city=table...|
|       1|        temp|       -148|     2|table1.city=table...|
|       1|        temp|       -148|     7|table1.city=table...|
|       1|        temp|        -44|     2|table1.city=table...|
|       1|        temp|        -44|     7|table1.city=table...|
|       1|        temp|        -60|     2|table1.city=table...|
|       1|        temp|        -60|     7|table1.city=table...|
|       1|        temp|        -75|     2|table1.city=table...|
|       1|        temp|        -75|     7|table1.city=table...|
|       1|        temp|        -86|     2|table1.city=table...|
|       1|    

Execution time is :

In [9]:
print("--- %s seconds ---" % (finish_time_dd-start_time_dd))

--- 8.5857129097 seconds ---


Because we have just one dc in all value in dc column is same